# GRAVITATIONAL LENS SDSSJ0819+5356

### PSF and aperture photometry, GALSIM

In [ ]:
import utilfunctions as uf
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib.dates as mdates
from matplotlib import ticker
import numpy as np
import warnings
import cupy as cp
# Packages used
import astropy.wcs as wcs
from astropy.coordinates import SkyCoord
from astropy.stats import mad_std
from astropy.wcs import WCS
import glob
import os
import pandas as pd
import copy
import pyregion
import scipy as sp
from photutils.background import MedianBackground, MeanBackground, ModeEstimatorBackground, MMMBackground, SExtractorBackground, BiweightLocationBackground
from photutils.background import Background2D
from photutils.detection import DAOStarFinder
from photutils.aperture import aperture_photometry, CircularAperture
from astropy.nddata.utils import Cutout2D
from photutils.psf import IntegratedGaussianPRF, SourceGrouper
from photutils.background import MMMBackground, LocalBackground
from photutils.psf import IterativePSFPhotometry, GriddedPSFModel
from astropy.modeling.fitting import LevMarLSQFitter
from astropy.table import QTable
import datetime
import math
from itertools import combinations
import PythonPhot as pp

In [ ]:
images_paths_QHY, images_paths_iKon, images_paths_all = uf.images_paths("SDSSJ0819+5356", directory_path = "work/red")

In [ ]:
# Components coordinates
coordsA=[124.998243440, 53.940423310]#A
coordsB=[125.00007152, 53.93966377] #B ra_decB = w_image.pixel_to_world( xA+3.367/0.507, yA+2.226/0.507)
coordsG=[124.999185780, 53.940057240] #G

# _______________________________________________________________________
### GALSIM

In [ ]:
import galsim
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt

# Define galaxy parameters
gal_x = 597.3053903003928   
gal_y = 1064.9221439339417
mag = 18.01
half_light_radius = 5.700197238658777  # in pixels
sersic_index = 6.2  # Adjusted to be within the valid range
axis_ratio = 0.7500
position_angle =  -35.5  # degrees

# Calculate flux from magnitude (assuming a zero point of 25.0 for simplicity)
zero_point = 24.5258  
flux = 10**(-0.4 * (mag - zero_point))

# Create the Sersic galaxy profile
gal = galsim.Sersic(n=sersic_index, half_light_radius=half_light_radius, flux=flux)
gal = gal.shear(q=axis_ratio, beta=position_angle * galsim.degrees)

# Create an image with a sky background
image_size = 2048  # Reduced image size to avoid memory issues
pixel_scale = 0.507  # arcseconds per pixel

# Define bounds to set the galaxy at the specific position within a smaller image
bounds = galsim.BoundsI(1, image_size, 1, image_size)
image = galsim.ImageF(bounds, scale=pixel_scale)
offset_x = gal_x - image_size // 2
offset_y = gal_y - image_size // 2
offset = galsim.PositionD(offset_x, offset_y)

# Draw the galaxy at the specified position
gal.drawImage(image=image, method='real_space', offset=offset)

# Add sky background
sky_background = 472.987876  # ADUs
image += sky_background

# Save the image to a FITS file
image.write("complex_galaxy.fits")

# Load and display the image
image_data = fits.getdata("complex_galaxy.fits")
plt.imshow(image_data, origin='lower')#, cmap='gray', vmin=sky_background, vmax=sky_background + flux / 10)
plt.colorbar()
plt.xlim(gal_x-50, gal_x+50)
plt.ylim(gal_y-50, gal_y+50)
plt.title("Simulated Galaxy Image")
plt.show()

print("Galaxy image created and saved as 'complex_galaxy.fits'")


In [ ]:
import galsim
import numpy as np

# Define the parameters for the galaxy
sersic_index = 6.2
half_light_radius = 5.700197238658777
axis_ratio = 0.75
angle = -35.5 * galsim.degrees
magnitude = 18.01
zero_point = 24.5258
pixel_scale = 0.507
sky_level = 0#472.987876  # in ADU

# Galaxy coordinates in a 2048x2048 image
gal_x = 597.272008553425
gal_y = 1064.9829494839112
print(gal_x,gal_y)
with fits.open(images_paths_iKon[100], memmap=True) as hdu:  
    image, w_image, header, dateob, filter, fwhm, sky, EXPT1 = uf.getinfo(hdu)
    print(len(image), images_paths_iKon[100] )
    
coords = [124.998243440, 53.940423310]
x0, y0 = uf.radec_to_xy(coords,  w_image)
gal_x=x0+(1.980/0.507)
gal_y=y0+(1.348/0.507)
print(gal_x,gal_y)


# Create the Sersic galaxy profile
galaxy = galsim.Sersic(n=sersic_index, half_light_radius=half_light_radius)
galaxy = galaxy.shear(q=axis_ratio, beta=angle)

# Adjust the galaxy flux to match the given magnitude
galaxy = galaxy.withFlux(10**(-0.4 * (magnitude - zero_point)))

# Load your custom PSF (assuming you have the PSF as an image file)
psf_image = galsim.fits.read("PSFs/psfTTT1_iKon936-1_2023-11-19-02-58-58-157009_SDSSJ0819+5356.fits")
psf = galsim.InterpolatedImage(psf_image)

# Convolve the galaxy with the PSF
convolved_galaxy = galsim.Convolve([galaxy, psf])

# Create a 2048x2048 image
image = galsim.ImageF(2048, 2048, scale=pixel_scale)

# Draw the convolved galaxy profile onto the image at the specified coordinates
bounds = galsim.BoundsI(int(gal_x - 100), int(gal_x + 100), int(gal_y - 100), int(gal_y + 100))
sub_image = image[bounds]
convolved_galaxy.drawImage(image=sub_image, offset=galsim.PositionD(gal_x % 1 - 0.5, gal_y % 1 - 0.5))

# Add sky background
image += sky_level

# Save the image to a file
image.write('simulated_galaxy_with_custom_psf.fits')

# Load and display the image
image_data = fits.getdata('simulated_galaxy_with_custom_psf.fits')
plt.imshow(image_data, origin='lower')#, cmap='gray', vmin=sky_background, vmax=sky_background + flux / 10)
plt.colorbar()
plt.xlim(gal_x-50, gal_x+50)
plt.ylim(gal_y-50, gal_y+50)
plt.title("Simulated Galaxy Image")
plt.show()

print("Simulation complete and image saved.")

# _______________________________________________________________________
### SERSIC model 

In [ ]:
import numpy as np
from scipy import ndimage
from astropy.io import fits
import matplotlib.pyplot as plt

def sersic_profile(x, y, x0, y0, n, Re, Ie, q, pa):
    """Generate a Sersic profile."""
    pa_rad = np.deg2rad(pa)
    cos_pa = np.cos(pa_rad)
    sin_pa = np.sin(pa_rad)
    x_prime = (x - x0) * cos_pa + (y - y0) * sin_pa
    y_prime = -(x - x0) * sin_pa + (y - y0) * cos_pa
    r = np.sqrt((x_prime**2 + (y_prime/q)**2))
    bn = 2 * n - 0.327
    return Ie * np.exp(-bn * ((r / Re)**(1/n) - 1))

# Define the parameters
sersic_index = 6.2
half_light_radius = 5.700197238658777
axis_ratio = 0.75
position_angle = -35.5
magnitude = 18.01
zero_point = 24.5258
pixel_scale = 0.507
sky_level = 472.987876  # in ADU
image_size = 2048

    
# Galaxy coordinates in the image
gal_x = 597.272008553425
gal_y = 1064.9829494839112

# Grid for the image
y, x = np.mgrid[0:image_size, 0:image_size]

# Calculate the effective intensity Ie
Ie = 10**(-0.4 * (magnitude - zero_point)) / (2 * np.pi * half_light_radius**2 * axis_ratio * sersic_index)

# Generate the Sersic profile
galaxy_image = sersic_profile(x, y, gal_x, gal_y, sersic_index, half_light_radius, Ie, axis_ratio, position_angle)

# Load and normalize the PSF image
psf_image = fits.getdata("PSFs/psfTTT1_iKon936-1_2023-11-19-02-58-58-157009_SDSSJ0819+5356.fits")
psf_image = psf_image / np.sum(psf_image)

# Convolve the galaxy profile with the PSF
convolved_image = ndimage.convolve(galaxy_image, psf_image, mode='constant')

# Scale the galaxy to be brighter than the sky
galaxy_max = np.max(convolved_image)
sky_max = sky_level
convolved_image *= (sky_max / galaxy_max)

# Add sky background
final_image = convolved_image + sky_level

# Save the final image to a FITS file
hdu = fits.PrimaryHDU(final_image)
hdu.writeto('simulated_galaxy_without_galsim.fits', overwrite=True)

# Display the final image
plt.imshow(final_image, origin='lower')
plt.colorbar()
plt.xlim(gal_x-50, gal_x+50)
plt.ylim(gal_y-50, gal_y+50)
plt.title('Simulated Galaxy with Custom PSF')
plt.show()

print("Simulation complete and image saved.")

In [ ]:
with fits.open(images_paths_iKon[100], memmap=True) as hdu:  
    image, w_image, header, dateob, filter, fwhm, sky, EXPT1 = uf.getinfo(hdu)
    print(len(image), images_paths_iKon[100] )

In [ ]:
# Load and display the image

fig, axs = plt.subplots(1, 3, figsize=(18, 6))

im0 = axs[0].imshow(image.get(), vmin=0, vmax=1050, cmap='viridis')
axs[0].set_title('Original Image')
axs[0].set_xlabel('X [pixels]')
axs[0].set_ylabel('Y [pixels]')
#axs[0].plot(gal_x, gal_y)
# axs[0].plot(597.3053903003928-554, 1064.9221439339417-1019, "x")
cbar0 = fig.colorbar(im0, ax=axs[0])
axs[0].set_xlim(635, 554)
axs[0].set_ylim(1100, 1019)
cbar0.set_label('Intensity')

# Model
im1 = axs[1].imshow(image_data[1019:1100,554:635], origin='lower', cmap='viridis')
axs[1].set_title('MODEL')
axs[1].set_xlabel('X [pixels]')
axs[1].set_ylabel('Y [pixels]')
# axs[1].plot(593.174812838104-554, 1062.6750714184982-1019, "x")
cbar1 = fig.colorbar(im1, ax=axs[1])
cbar1.set_label('Intensity')

# Model
im2 = axs[2].imshow((image.get()-image_data),vmin=0, vmax=1050,  cmap='viridis')
axs[2].set_title('REST')
axs[2].set_xlabel('X [pixels]')
axs[2].set_ylabel('Y [pixels]')
axs[2].set_xlim(635, 554)
axs[2].set_ylim(1100, 1019)
# axs[2].plot(593.174812838104-554, 1062.6750714184982-1019, "x") -2*image_data)[1019:1100,554:635]
cbar2 = fig.colorbar(im2, ax=axs[2])
cbar2.set_label('Intensity')



# _______________________________________________________________________
### APERTURE Photometry

In [ ]:
from astropy.visualization import simple_norm
import matplotlib.pyplot as plt
from photutils.detection import DAOStarFinder
from photutils.aperture import CircularAperture
from photutils.aperture import aperture_photometry
coords = cp.asarray([[125.037078920, 53.968357790], [125.013046340, 53.934497430], [125.012248390, 53.931427250], [124.998243440, 53.940423310], [124.998243440, 53.940423310],[125.010332700, 53.927527680]]) 
#coords = cp.asarray([[124.998243440, 53.940423310],[124.998243440, 53.940423310],[125.037078920, 53.968357790], [125.013046340, 53.934497430], [125.012248390, 53.931427250], [125.010332700, 53.927527680], [124.985519970, 53.943565380], [125.024931110, 53.921033100], [124.985143180, 53.950553540], [125.023014140, 53.950554040]]) # A, B, D, E, F, G, H, I, J, K
coordsstars = cp.asarray([[125.037078920, 53.968357790], [125.013046340, 53.934497430], [125.010332700, 53.927527680], [125.024931110, 53.921033100], [125.023014140, 53.950554040]]) # D, E, G, I, K

estrella1=[]
estrella2=[]
estrella3=[]
estrella4=[]
A=[]
B=[]
time=[]
filtro=[]
fwhms=[]
count=0
for i in images_paths_iKon:
    with fits.open(i, memmap=True) as hdu:
        try: 
            image, w_image, header, dateob, filter, fwhm, sky, EXPT1 = uf.getinfo(hdu)
            x0, y0 = uf.radec_to_xy(coordsstars[0].get(), w_image)
            x1, y1 = uf.radec_to_xy(coordsstars[1].get(), w_image)
            x2, y2 = uf.radec_to_xy(coordsstars[2].get(), w_image)
            x3, y3 = uf.radec_to_xy(coordsstars[3].get(), w_image)
            x4, y4 = uf.radec_to_xy(coordsstars[4].get(), w_image)
            try:
                
                sources = np.array([[x0,x1,x2,x3,x4],[y0,y1,y2,y3,y4]])
                positions = np.transpose((sources[0], sources[1]))
                
                apertures = CircularAperture(positions, r=0.5)
                
                phot_table = aperture_photometry(image.get(), apertures)
                if count==0:
                    print(phot_table)
                    count+=1
                estrella1.append(phot_table["aperture_sum"][0])
                estrella2.append(phot_table["aperture_sum"][1])
                estrella3.append(phot_table["aperture_sum"][2])
                estrella4.append(phot_table["aperture_sum"][3])
                
                #A.append(phot_table["aperture_sum"][3])
                B.append(phot_table["aperture_sum"][4])
                time.append(dateob)
                filtro.append(filter)
                fwhms.append(fwhm)
            
            except:
                continue
        except:
                continue
#print(len(estrella1),len(estrella2),len(A),len(B),len(time), len(filtro))

In [ ]:
diferent_filters = np.unique(filtro)
print(diferent_filters)
goodseeing = np.where(np.array(fwhms) < (50/0.508))[0]

estrella12=[np.array([]) for i in range(len(diferent_filters))]
estrella22=[np.array([]) for i in range(len(diferent_filters))]
estrella32=[np.array([]) for i in range(len(diferent_filters))]
estrella42=[np.array([]) for i in range(len(diferent_filters))]
A2=[np.array([]) for i in range(len(diferent_filters))]
B2=[np.array([]) for i in range(len(diferent_filters))]
time2 = [np.array([]) for i in range(len(diferent_filters))]
for i in range(len(diferent_filters)):
    index = np.where(np.array(filtro) == diferent_filters[i])[0]
    for j in index:
        if j in goodseeing:   
            estrella12[i] = np.append(estrella12[i], estrella1[j])
            estrella22[i] = np.append(estrella22[i], estrella2[j])
            estrella32[i] = np.append(estrella32[i], estrella3[j])
            estrella42[i] = np.append(estrella42[i], estrella4[j])
            #A2[i] = np.append(A2[i], A[j])
            B2[i] = np.append(B2[i], B[j])
            time2[i] = np.append(time2[i], time[j])


In [ ]:
a=[50,50,50,50,50,50,50]
figure, axis = plt.subplots(1, 6,figsize=(50, 10))
figure.tight_layout(pad=3.5)
dif=[np.asarray([]) for i in range(len(estrella12))]
for i in range(len(estrella12)):
    for j, f in enumerate(estrella12[i]):
        dif[i] = np.append(dif[i], -2.5*np.log10(f/estrella42[i][j]))
    difmag = -2.5*np.log10(np.array(estrella12[i])/np.array(estrella42[i]))
    difmag2 = -2.5*np.log10(np.array(estrella22[i])/np.array(estrella42[i]))
    difmag3 = -2.5*np.log10(np.array(estrella32[i])/(np.array(estrella12[i])+np.array(estrella22[i])+np.array(estrella42[i])))
    difmag4 = -2.5*np.log10(np.array(B2[i])/(np.array(estrella12[i])+np.array(estrella22[i])+np.array(estrella32[i])+np.array(estrella42[i])))
#plt.errorbar(time2[0],difmag,yerr=np.std((difmag[np.logical_not(np.isnan(difmag))])), elinewidth=0.75, linewidth=0,   marker=".",label = f"σ = {np.std((difmag[np.logical_not(np.isnan(difmag))]))}")
#plt.errorbar(time2[0],difmag2,yerr=np.std((difmag2[np.logical_not(np.isnan(difmag2))])), elinewidth=0.75, linewidth=0,   marker=".",label = f"σ = {np.std((difmag2[np.logical_not(np.isnan(difmag2))]))}")
    axis[i].errorbar(time2[i], dif[i], yerr=0, elinewidth=0.75, linewidth=0,   marker=".", label = f"σ = {np.std((difmag[np.logical_not(np.isnan(difmag3))]))}")
    #axis[i].errorbar(time2[i], difmag2, yerr=0, elinewidth=0.75, linewidth=0,   marker=".",label = f"σ = {np.std((difmag2[np.logical_not(np.isnan(difmag4))]))}")
    #axis[i].errorbar(time2[i], difmag3, yerr=0, elinewidth=0.75, linewidth=0,   marker=".",label = f"σ = {np.std((difmag3[np.logical_not(np.isnan(difmag4))]))}")
    #axis[i].errorbar(time2[i], difmag4, yerr=0, elinewidth=0.75, linewidth=0,   marker=".",label = f"σ = {np.std((difmag4[np.logical_not(np.isnan(difmag4))]))}")
    axis[i].legend()
#plt.savefig('galfit'+'.png')

# _______________________________________________________________________
### PSF Photometry

In [ ]:
# PSF PHOTOMETRY USING DAOPHOT

warnings.filterwarnings("ignore") # Not showing warmings

image_paths = images_paths_iKon

coords = cp.asarray([[124.998243440, 53.940423310],[124.998243440, 53.940423310],[125.037078920, 53.968357790], [125.013046340, 53.934497430], [125.010332700, 53.927527680], [125.024931110, 53.921033100], [125.023014140, 53.950554040]]) # D, E, G, I, K
#coords = cp.asarray([[124.998243440, 53.940423310],[124.998243440, 53.940423310],[125.037078920, 53.968357790], [125.013046340, 53.934497430], [125.012248390, 53.931427250], [125.010332700, 53.927527680], [124.985519970, 53.943565380], [125.024931110, 53.921033100], [124.985143180, 53.950553540], [125.023014140, 53.950554040]]) # A, B, D, E, F, G, H, I, J, K
coordsstars = cp.asarray([[124.998243440, 53.940423310],[124.998243440, 53.940423310],[125.037078920, 53.968357790], [125.013046340, 53.934497430], [125.010332700, 53.927527680], [125.024931110, 53.921033100], [125.023014140, 53.950554040]]) # D, E, G, I, K

# Components coordinates
coordsA=[124.998243440, 53.940423310]#A
coordsB=[125.00007152, 53.93966377] #B ra_decB = w_image.pixel_to_world( xA+3.367/0.507, yA+2.226/0.507)
coordsG=[124.999185780, 53.940057240] #G


bkg_estimator = LocalBackground(5, 10 , MMMBackground())
psf_model = IntegratedGaussianPRF()
daogroup = SourceGrouper(2.0)  # Group stars that are closer than 2 pixels
fitter = LevMarLSQFitter()

images_with_a_KeyErrorstars = np.array([])
images_other_errorstars = np.array([])
filters = np.array([])
dateobs = np.array([])
fwhms = np.array([])
flux = [np.asarray([]) for i in range(len(coordsstars))]  # A, B, D, E, F, G, H, I, J, K
eflux = [np.asarray([]) for i in range(len(coordsstars))]  # A, B, D, E, F, G, H, I, J, K
EXPTs = np.array([])
count=0
for i in image_paths:
    
    with fits.open(i, memmap=True) as hdu:
        try:
            
            image, w_image, header, dateob, filter, fwhm, sky, EXPT1 = uf.getinfo(hdu)
            daofind = DAOStarFinder(fwhm=fwhm, threshold=50) 
            photometry = IterativePSFPhotometry(
            finder=daofind,
            grouper=daogroup,
            psf_model=psf_model,
            localbkg_estimator=bkg_estimator,
            fitter=fitter,
            maxiters=1,
            fit_shape=(11, 11),
            aperture_radius=1.0)
            

            if count==100:
                daofind = DAOStarFinder(fwhm=fwhm, threshold=100)
                print(sky)
                sources = daofind(image.get())
                print("hola")
                
            for j in range(len(coordsstars)):
                
                if j==1:
                    x1, y1 = uf.radec_to_xy(coordsstars[j].get(), w_image)
                    init_params = QTable()
                    init_params['x'] = [x1+(3.367/0.507)]
                    init_params['y'] = [y1+(2.226/0.507)]
                else:
                    x1, y1 = uf.radec_to_xy(coordsstars[j].get(), w_image)
                    init_params = QTable()
                    init_params['x'] = [x1]
                    init_params['y'] = [y1]
                
                #print(photometry(image.get(), init_params=init_params))
                result_tab = photometry(image.get(), init_params=init_params)
                #print(result_tab)
                result_tab = result_tab.to_pandas()
                
                flux[j] = np.append(flux[j], result_tab["flux_fit"][0])
                eflux[j] = np.append(eflux[j], result_tab["flux_err"][0])
                #if i == images_paths_iKon[100] and j==3:
                    #plt.imshow(image.get(), vmin=200, vmax=1000)
                    #plt.plot( x1, y1, "x", mec='r', ms=50)
                    #plt.plot( x1-3.367/0.508, y1-2.226/0.508, "x", mec='r', ms=50)
                    #plt.plot( result_tab["x_fit"], result_tab["y_fit"], "x", mec='b', ms=50)
                    #plt.plot(x1-(1.980/0.508),y1-(1.348/0.508), "x", mec='g', ms=50)
                    #plt.xlim( x1+20, x1-20)
                    #plt.ylim( y1+20, y1-20)
                    #print(result_tab)
            filters = np.append(filters, filter)
            dateobs = np.append(dateobs, dateob)
            fwhms = np.append(fwhms, fwhm)
            EXPTs = np.append(EXPTs, EXPT1)
            
        except KeyError:
            images_with_a_KeyErrorstars = np.append(images_with_a_KeyErrorstars, i)

        except:
            images_other_errorstars = np.append(images_other_errorstars, i)
            
    count+=1
    
print(len(images_paths_iKon), len(images_with_a_KeyErrorstars), len(images_other_errorstars))

In [ ]:
# Function for differential photometry between stars with the flux and the dateobs in arrays
def star_diff_phot(fluxstars, dateobs, diferent_filters, starsname = False): 
    # fluxstars: array separated by filters, in each filter the flux values of each star in different arrays, for a star each value of flux corresponds to a different image.
    # dateobs: array separated by filters, each date corresponds to a different image.
    # starsname: array with names of the stars, default the stars are named with numbers.
    # diferent_filters: array with the filters
    
    if starsname == False: # Stars named with numbers
        starsname = [str(i) for i in range(len(fluxstars[0]))]
    
    diff_phot = {} # Diff. phot values
    diff_phot_names = {} # Diff. phot star combinations names
    diff_phot_dates = {} # Dates for each diff. phot value
    
    for num_stars in range(1, len(starsname)): # Differential photometry using different numbers of stars like reference stars
        diff_phot[num_stars] = {} 
        diff_phot_names[num_stars] = {}
        diff_phot_dates[num_stars] = {}
                           
        for i, flux_star in enumerate(fluxstars):  # Separated for each filter
            diff_phot[num_stars]['filter_' + diferent_filters[i]] = []
            diff_phot_names[num_stars]['filter_' + diferent_filters[i]] = []
            diff_phot_dates[num_stars]['filter_' + diferent_filters[i]] = []  
            
            for j, star_flux in enumerate(flux_star): # Differential photometry of one star in front of the rest
                for combo in combinations(np.delete(range(len(flux_star)), j), num_stars): # Combinations of the stars used like reference stars
                    diffphot = -2.5 * np.log10(sum(flux_star[k] for k in combo) / star_flux ) #  sum(flux_star[k] for k in combo) / star_flux
                    diff_phot[num_stars]['filter_' + diferent_filters[i]].append(diffphot)
                    diff_phot_names[num_stars]['filter_' + diferent_filters[i]].append(starsname[j] + ''.join(starsname[k] for k in combo))
                    diff_phot_dates[num_stars]['filter_' + diferent_filters[i]].append(dateobs[i])

    return diff_phot, diff_phot_names, diff_phot_dates 

In [ ]:
def error_diff_phot(fluxstars, errorfluxstars, diferent_filters, starsname = False):
    if starsname == False: # Stars named with numbers
        starsname = [str(i) for i in range(len(fluxstars[0]))]
    
    error = {}
    errordiff_phot_names = {}
    for num_stars in range(1, len(starsname)): # Differential photometry using different numbers of stars like reference stars
        error[num_stars] = {} 
        errordiff_phot_names[num_stars] = {}                  
        for i, flux_star in enumerate(fluxstars):  # Separated for each filter
            error[num_stars]['filter_' + diferent_filters[i]] = []
            errordiff_phot_names[num_stars]['filter_' + diferent_filters[i]] = []
            for j, star_flux in enumerate(flux_star): # Differential photometry of one star in front of the rest
                for combo in combinations(np.delete(range(len(flux_star)), j), num_stars): # Combinations of the stars used like reference stars
                    errordiffphot = 2.5 * np.log10(np.e) * np.sqrt(sum((errorfluxstars[i][k]**2) for k in combo)/((sum(flux_star[k] for k in combo))**2) + (errorfluxstars[i][j]/star_flux)**2) 
                    error[num_stars]['filter_' + diferent_filters[i]].append(errordiffphot)
                    errordiff_phot_names[num_stars]['filter_' + diferent_filters[i]].append(starsname[j] + ''.join(starsname[k] for k in combo))

    return error, errordiff_phot_names                

    

In [ ]:
# Function for differential photometry between the components of the lens and the stars with the flux and the dateobs in arrays
def component_diff_phot(fluxstars, fluxcomp, dateobs, diferent_filters, starsname = False, compname = False): 
    # fluxstars: array separated by filters, in each filter the flux values of each star in different arrays, for a star each value of flux corresponds to a different image.
    # fluxcomp: array separated by filters, in each filter the flux values of each component in different arrays, for a component each value of flux corresponds to a different image.
    # dateobs: array separated by filters, each date corresponds to a different image.
    # starsname: array with names of the stars, default the stars are named with numbers.
    # compname: array with names of the components, default the components are named with numbers.
    # diferent_filters: array with the filters
    
    if starsname == False: # Stars named with numbers
        starsname = [str(i) for i in range(len(fluxstars[0]))]
        
    if compname == False: # Components named with numbers
        compname = [str(i) for i in range(len(fluxcomp[0]))]
    
    #diff_phot = {} # Diff. phot values
    #diff_phot_names = {} # Diff. phot components and stars combinations names
    #diff_phot_dates = {} # Dates for each diff. phot value

    diff_photA = {} # Diff. phot values
    diff_phot_namesA = {} # Diff. phot components and stars combinations names
    diff_phot_datesA = {} # Dates for each diff. phot value
    
    diff_photB = {} # Diff. phot values
    diff_phot_namesB = {} # Diff. phot components and stars combinations names
    diff_phot_datesB = {} # Dates for each diff. phot value
    
    for num_stars in range(1, len(starsname)+1): # Differential photometry using different numbers of stars like reference stars
        #diff_phot[num_stars] = {} 
        #diff_phot_names[num_stars] = {}
        #diff_phot_dates[num_stars] = {}

        diff_photA[num_stars] = {} 
        diff_phot_namesA[num_stars] = {}
        diff_phot_datesA[num_stars] = {}
        
        diff_photB[num_stars] = {} 
        diff_phot_namesB[num_stars] = {}
        diff_phot_datesB[num_stars] = {}
        
                           
        for i, flux_star in enumerate(fluxstars):  # Separated for each filter
            #diff_phot[num_stars]['filter_' + diferent_filters[i]] = []
            #diff_phot_names[num_stars]['filter_' + diferent_filters[i]] = []
            #diff_phot_dates[num_stars]['filter_' + diferent_filters[i]] = [] 

            diff_photA[num_stars]['filter_' + diferent_filters[i]] = []
            diff_phot_namesA[num_stars]['filter_' + diferent_filters[i]] = []
            diff_phot_datesA[num_stars]['filter_' + diferent_filters[i]] = []  
            
            diff_photB[num_stars]['filter_' + diferent_filters[i]] = []
            diff_phot_namesB[num_stars]['filter_' + diferent_filters[i]] = []
            diff_phot_datesB[num_stars]['filter_' + diferent_filters[i]] = []
            
            for j, comp_flux in enumerate(fluxcomp[i]): # Differential photometry of one component in front of the stars
                for combo in combinations(range(len(flux_star)), num_stars): # Combinations of the stars used like reference stars
                    if j==0:
                        diffphotA = -2.5 * np.log10(comp_flux / sum(flux_star[k] for k in combo))
                        diff_photA[num_stars]['filter_' + diferent_filters[i]].append(diffphotA)
                        diff_phot_namesA[num_stars]['filter_' + diferent_filters[i]].append(compname[j] + ''.join(starsname[k] for k in combo))
                        diff_phot_datesA[num_stars]['filter_' + diferent_filters[i]].append(dateobs[i])
                        
                    else:
                        
                        diffphotB = -2.5 * np.log10(comp_flux / sum(flux_star[k] for k in combo))  
                        diff_photB[num_stars]['filter_' + diferent_filters[i]].append(diffphotB)
                        diff_phot_namesB[num_stars]['filter_' + diferent_filters[i]].append(compname[j] + ''.join(starsname[k] for k in combo))
                        diff_phot_datesB[num_stars]['filter_' + diferent_filters[i]].append(dateobs[i])
                        
                    #diffphot = -2.5 * np.log10(comp_flux / sum(flux_star[k] for k in combo)) 
                    #diff_phot[num_stars]['filter_' + diferent_filters[i]].append(diffphot)
                    #diff_phot_names[num_stars]['filter_' + diferent_filters[i]].append(compname[j] + ''.join(starsname[k] for k in combo))
                    #diff_phot_dates[num_stars]['filter_' + diferent_filters[i]].append(dateobs[i])

                    

    return diff_photA, diff_phot_namesA, diff_phot_datesA, diff_photB, diff_phot_namesB, diff_phot_datesB

In [ ]:
# Function for compute the scale factors for the σ of differential photometry between components and stars
def scalefactors(fluxstars, errorfluxstars, fluxcomp, errorfluxcomp, diferent_filters, starsname = False):
    # fluxstars: array separated by filters, in each filter the flux values of each star in different arrays, for a star each value of flux corresponds to a different image.
    # fluxcomp: array separated by filters, in each filter the flux values of each component in different arrays, for a component each value of flux corresponds to a different image.
    # diferent_filters: array with the filters
    # starsname: array with names of the stars, default the stars are named with numbers.
    # compname: array with names of the components, default the components are named with numbers.
    # diferent_filters: array with the filters
    
    if starsname == False: # Stars named with numbers
        starsname = [str(i) for i in range(len(fluxstars[0]))]
    
    #scale_factors = {} # Scale factors (SF) -> σ_comp = SF * σ_stars_after_plot
    #scale_factors_names = {} # To know which components and which stars have been used 

    scale_factorsA = {} # Scale factors (SF) -> σ_comp = SF * σ_stars_after_plot
    scale_factorsB = {} 
    scale_factors_namesA = {} # To know which components and which stars have been used 
    scale_factors_namesB = {}
    
    for num_stars in range(1, len(starsname)): # Number of reference stars
        #scale_factors[num_stars] = {}
        #scale_factors_names[num_stars] = {}

        scale_factorsA[num_stars] = {}
        scale_factors_namesA[num_stars] = {}
        scale_factorsB[num_stars] = {}
        scale_factors_namesB[num_stars] = {}
        
        for i, flux_star in enumerate(fluxstars): # Separated for each filter
            #scale_factors[num_stars]['filter_' + diferent_filters[i]] = []
            #scale_factors_names[num_stars]['filter_' + diferent_filters[i]] = []

            scale_factorsA[num_stars]['filter_' + diferent_filters[i]] = []
            scale_factors_namesA[num_stars]['filter_' + diferent_filters[i]] = []
            scale_factorsB[num_stars]['filter_' + diferent_filters[i]] = []
            scale_factors_namesB[num_stars]['filter_' + diferent_filters[i]] = []
            
            for j, comp_flux in enumerate(fluxcomp[i]):
                for combo in combinations(range(len(flux_star)), num_stars): # Implementation the sum of stars' fluxes
                    for s in range(len(flux_star)): 
                        if s not in combo: # Stars no used in the combo to compute the σ_stars_from_flux_error needed for SF = σ_stars_from_flux_error/σ_comp_from_flux_error
                            #scale_factors[num_stars]['filter_' + diferent_filters[i]].append((flux_star[s] / comp_flux) ** 2 * (
                                        #((sum(flux_star[k] for k in combo))**2 * (errorfluxcomp[i][j])**2 +
                                         #(comp_flux)**2 * (sum(errorfluxstars[i][k] for k in combo))**2) /
                                        #(flux_star[s]** 2 * (sum(errorfluxstars[i][k] for k in combo))**2 +
                                         #errorfluxstars[i][s]** 2 * (sum(flux_star[k] for k in combo))**2)))
                            #if j == 0: # A
                                #scale_factors_names[num_stars]['filter_' + diferent_filters[i]].append("A" + starsname[s] + ''.join(starsname[k] for k in combo))
                            #else: # B
                                #scale_factors_names[num_stars]['filter_' + diferent_filters[i]].append("B" + starsname[s] + ''.join(starsname[k] for k in combo))   
                            if j == 0: # A
                                scale_factorsA[num_stars]['filter_' + diferent_filters[i]].append((flux_star[s] / comp_flux) ** 2 * (
                                        ((sum(flux_star[k] for k in combo))**2 * (errorfluxcomp[i][j])**2 +
                                         (comp_flux)**2 * (sum(errorfluxstars[i][k] for k in combo))**2) /
                                        (flux_star[s]** 2 * (sum(errorfluxstars[i][k] for k in combo))**2 +
                                         errorfluxstars[i][s]** 2 * (sum(flux_star[k] for k in combo))**2)))
                                scale_factors_namesA[num_stars]['filter_' + diferent_filters[i]].append("A" + starsname[s] + ''.join(starsname[k] for k in combo))
                            
                            else: # B  
                                scale_factorsB[num_stars]['filter_' + diferent_filters[i]].append((flux_star[s] / comp_flux) ** 2 * (
                                        ((sum(flux_star[k] for k in combo))**2 * (errorfluxcomp[i][j])**2 +
                                         (comp_flux)**2 * (sum(errorfluxstars[i][k] for k in combo))**2) /
                                        (flux_star[s]** 2 * (sum(errorfluxstars[i][k] for k in combo))**2 +
                                         errorfluxstars[i][s]** 2 * (sum(flux_star[k] for k in combo))**2)))
                                scale_factors_namesB[num_stars]['filter_' + diferent_filters[i]].append("B" + starsname[s] + ''.join(starsname[k] for k in combo))
    return scale_factorsA, scale_factors_namesA, scale_factorsB, scale_factors_namesB
    #return scale_factors, scale_factors_names

In [ ]:
diferent_filters = np.unique(filters)
goodseeing = np.where(fwhms < (50/0.508))[0]

fwhms2 = [np.array([]) for i in range(len(diferent_filters))]
dateobs2 = [np.array([]) for i in range(len(diferent_filters))]
EXPTs2 = [np.array([]) for i in range(len(diferent_filters))]


indexfilters = [np.array([]) for i in range(len(diferent_filters))]

for i in range(len(diferent_filters)):
    index = np.where(filters == diferent_filters[i])[0]
    
    for j in index:
        
        if j in goodseeing:       
        
            fwhms2[i] = np.append(fwhms2[i], fwhms[j])
            dateobs2[i] = np.append(dateobs2[i], dateobs[j])
            EXPTs2[i] = np.append(EXPTs2[i], EXPTs[j])
            
            indexfilters[i] = np.append(indexfilters[i], j)

#print(indexfilters)
# Group the fluxes in stars' fluxes and components' fluxes and group them in diferent filter. Also the errors.
errorfluxstars0=[]
fluxstars0=[]
fluxcomp0=[]
errorfluxcomp0=[]

for i in range(len(diferent_filters)):
    errorinter = [np.array([]) for i in range(len(flux)-2)]
    inter = [np.array([]) for i in range(len(flux)-2)]
    intercomp = [np.array([]) for i in range(2)]
    errorintercomp = [np.array([]) for i in range(2)]
    # totes les estrelles per cada filtre
    for j in range(len(flux)):
        if j<2:
            for k in indexfilters[i]:
                intercomp[j] = np.append(intercomp[j], flux[j][int(k)])
                errorintercomp[j] = np.append(errorintercomp[j], eflux[j][int(k)])
            #for k in indexfilters[i]:
                #errorinter[j] = np.append(errorinter[j], eflux[j][int(k)])
                #inter[j] = np.append(inter[j], flux[j][int(k)])
                
        else:
            for k in indexfilters[i]:
                errorinter[j-2] = np.append(errorinter[j-2], eflux[j][int(k)])
                inter[j-2] = np.append(inter[j-2], flux[j][int(k)])
            #for k in indexfilters[i]:
                #intercomp[j-2] = np.append(intercomp[j-2], flux[j][int(k)])
                #errorintercomp[j-2] = np.append(errorintercomp[j-2], eflux[j][int(k)])
                
    errorfluxstars0.append(errorinter)
    fluxstars0.append(inter) #afegim filtres amb les estrelles separades
    fluxcomp0.append(intercomp)
    errorfluxcomp0.append(errorintercomp)


In [ ]:
import copy
fluxstars = copy.deepcopy(fluxstars0)
errorfluxstars = copy.deepcopy(errorfluxstars0)
fluxcomp = copy.deepcopy(fluxcomp0)
errorfluxcomp = copy.deepcopy(errorfluxcomp0)

In [ ]:
# With the fluxes from the psf photometry, differential photometry and scale factor calculations
compname = ["A","B"]
starsname = ["D", "E", "G", "I", "K"]
diff_phot_compA, diff_phot_names_compA, diff_phot_dates_compA, diff_phot_compB, diff_phot_names_compB, diff_phot_dates_compB = component_diff_phot(fluxstars, fluxcomp, dateobs2, diferent_filters, starsname, compname)
diff_phot, diff_phot_names, diff_phot_dates = star_diff_phot(fluxstars, dateobs2, diferent_filters, starsname)
scale_factorsA, scale_factors_namesA, scale_factorsB, scale_factors_namesB = scalefactors(fluxstars, errorfluxstars, fluxcomp, errorfluxcomp, diferent_filters, starsname)
error, errordiff_phot_names = error_diff_phot(fluxstars, errorfluxstars, diferent_filters, starsname)

In [ ]:
a=[250,250,250,250,250,250,250,250,250,250,250]
for o, t in enumerate(diff_phot):
    figure, axis = plt.subplots(len(diff_phot[t]), len(diff_phot[t]["filter_Lum"]),figsize=(a[o], 60))
    figure.tight_layout(pad=3.5)
    for i, filtre in enumerate(diff_phot[t]):
        for j, phot in enumerate(diff_phot[t][filtre]):
        
            axis[i][j].errorbar(diff_phot_dates[t]['filter_' + diferent_filters[i]][j]-2460000.0, phot, yerr=np.std((phot[np.logical_not(np.isnan(phot))])), elinewidth=0.75, linewidth=0,   marker=".", label = f"σ = {np.std((phot[np.logical_not(np.isnan(phot))]))}")# + r' $\bar{σ}$ = ' + f"{np.mean(error[t]['filter_' + diferent_filters[i]][j][np.logical_not(np.isnan(error[t]['filter_' + diferent_filters[i]][j]))])}")
            axis[i][j].legend()
            #axis[i][j].set_ylim([np.mean(group3_2[i][j][np.logical_not(np.isnan(group3_2[i][j]))])-10*np.std(group3_2[i][j][np.logical_not(np.isnan(group3_2[i][j]))]),np.mean(group3_2[i][j][np.logical_not(np.isnan(group3_2[i][j]))])+10*np.std(group3_2[i][j][np.logical_not(np.isnan(group3_2[i][j]))])])
            axis[i][j].set(xlabel='Time JD-2460000', ylabel='Phot. Diff.')
            axis[i][j].set_title("Filter "+diferent_filters[i]+ " star " + diff_phot_names[t]['filter_' + diferent_filters[i]][j][0]+ " and ref. star " + diff_phot_names[t]['filter_' + diferent_filters[i]][j][1:] )
    # plt.show()
    # plt.savefig('diff_phot_stars'+str(t)+'.png')

In [ ]:
a=[500,500,500,500,500,500,500,500,500,500,500,500,500,500,500,500,500,500]
for o, t in enumerate(scale_factors_names):
    #print(o)
    #if o<2:
        figure, axis = plt.subplots(len(scale_factors_names[t]), len(scale_factors_names[t]["filter_Lum"]),figsize=(a[o], 60))
        figure.tight_layout(pad=3.5)

        for i, filtre in enumerate(scale_factors_names[t]):
            for j, l in enumerate(scale_factors_names[t][filtre]):
                for r, n in enumerate(diff_phot_names[t]['filter_' + diferent_filters[i]]):
                    if n==l[1:]:
                        for g, phot in enumerate(diff_phot_comp[t]['filter_' + diferent_filters[i]]):
                            #print(diff_phot_names_comp[t]['filter_' + diferent_filters[i]][g], n, l, phot )
                            if l[0]==diff_phot_names_comp[t]['filter_' + diferent_filters[i]][g][0] and diff_phot_names_comp[t]['filter_' + diferent_filters[i]][g][1:]==n[1:]: 

                                axis[i][j].errorbar(diff_phot_dates_comp[t]['filter_' + diferent_filters[i]][g]-2460000.0, phot, yerr=scale_factors[t][filtre][j]*np.std((diff_phot[t][filtre][r][np.logical_not(np.isnan(diff_phot[t][filtre][r]))])), elinewidth=0.75, linewidth=0,   marker=".", label = f"σ = $\Gamma ^2 $ x {np.std((diff_phot[t][filtre][r][np.logical_not(np.isnan(diff_phot[t][filtre][r]))]))}")
                                axis[i][j].legend()
                                axis[i][j].set(xlabel='Time JD-2460000', ylabel='Phot. Diff.')
                                axis[i][j].set_title("Filter "+diferent_filters[i] + " comp. "+ diff_phot_names_comp[t]['filter_' + diferent_filters[i]][g][0] + " and ref. star " + diff_phot_names_comp[t]['filter_' + diferent_filters[i]][g][1:] +", $\Gamma ^2 $ using"+ l +", σ_stars_"+ n )
        #print("good")
        #break
    #plt.show()

In [ ]:
diff_phot_dates2 = copy.deepcopy(diff_phot_dates)
error2 = copy.deepcopy(error)
for o, t in enumerate(diff_phot):
    std2=[]
    stdmeanpercentile=[]
    stdpercentile2=[]
    combos2sorted=[]
    for i, filters in enumerate(diff_phot[t]):
        if i!=0:
            stdinter=[]
            stdpercentileinter=[]
            stdmeaninter=[]
            #count=0
            for j, combo in enumerate(diff_phot[t][filters]):
                deletepositions2=[]
                deletepositions3=[]
                # for o, l in enumerate(np.logical_not(np.isnan(combo))):
                    # if l == np.bool_(False):
                        # diff_phot_dates2[t][filters][j] = np.delete(diff_phot_dates2[t][filters][j], o)
                        # error2[t]['filter_' + diferent_filters[i]][j] = np.delete(error2[t]['filter_' + diferent_filters[i]][j], o)
                combo=(combo)[np.logical_not(np.isnan(combo))]    
                err = error2[t]['filter_' + diferent_filters[i]][j]
                for k in combo:
                    if k < np.quantile(combo, 0.1) or k > np.quantile(combo, 0.9):     
                        deletepositions2.append(np.where(combo == k)[0][0])  
                for k in err:
                    if k < np.quantile(err, 0.3) or k > np.quantile(err, 0.7):     
                        deletepositions3.append(np.where(err == k)[0][0]) 
                combo = np.delete(combo, deletepositions2)
                
                err = np.delete(err, deletepositions2+deletepositions3)
                #err = np.delete(err, deletepositions3)
                err= err[np.logical_not(np.isnan(err))]
                stdinter.append(np.std(combo))
                stdpercentileinter.append(np.std(combo))
                
                stdmeaninter.append(np.mean(err))
                
            sortered = sorted(zip(stdpercentileinter,stdinter,diff_phot_names[t][filters],stdmeaninter))
            std2.append([x[1] for x in sortered])
            stdpercentile2.append([x[0] for x in sortered])
            combos2sorted.append([x[2] for x in sortered])
            stdmeanpercentile.append([x[3] for x in sortered])

    figure, axis = plt.subplots(1, len(stdpercentile2),figsize=(60, 10), layout='constrained')
    #figure.tight_layout(pad=3.5)
    for i in range(len(stdpercentile2)):
        x = np.arange(len(combos2sorted[i]))
        width = 0.3
        multiplier = 0
        #bars=axis[i].bar(combos3sorted[i], stdpercentile[i])
        roundvalues=[round(val, 4) for val in stdpercentile2[i]]
        roundvalues2=[round(val, 4) for val in stdmeanpercentile[i]]
        a=[stdpercentile2[i]]#, stdmeanpercentile[i]]
        b=[roundvalues]#, roundvalues2]
        #print(roundvalues2)
        for p in range(0,2):
            offset = width * multiplier
            axis[i].bar_label(axis[i].bar(x + offset, a[p], width), labels=  b[p], padding=3)
            multiplier += 1
        axis[i].set(xlabel='Star groups', ylabel='Standard deviation')
        axis[i].set_title("Filter "+ diferent_filters[i+1])
        axis[i].set_xticks(x, combos2sorted[i])
    
    #plt.savefig('standarddeviation+propagationerrorPSF'+str(t)+'.png')

    


In [ ]:

a=[250,400,250,100]
for o, t in enumerate(scale_factors_names):
    figure, axis = plt.subplots(len(scale_factors_names[t]), len(scale_factors_names[t]["filter_Lum"]),figsize=(a[o], 60))
    figure.tight_layout(pad=3.5)
    for i, filtre in enumerate(scale_factors_names[t]):
        for j, l in enumerate(scale_factors_names[t][filtre]):
            for r, n in enumerate(diff_phot_names[t]['filter_' + diferent_filters[i]]):
                if n==l[1:]:
                    for g, phot in enumerate(diff_phot_comp[t]['filter_' + diferent_filters[i]]):
                        #print(diff_phot_names_comp[t]['filter_' + diferent_filters[i]][g], n, l, phot )
                        if l[0]==diff_phot_names_comp[t]['filter_' + diferent_filters[i]][g][0] and diff_phot_names_comp[t]['filter_' + diferent_filters[i]][g][1:]==n[1:]: #scale_factors[t][filtre][j]*
                            
                            axis[i][j].errorbar(diff_phot_dates_comp[t]['filter_' + diferent_filters[i]][g]-2460000.0, phot, yerr=np.std((diff_phot[t][filtre][r][np.logical_not(np.isnan(diff_phot[t][filtre][r]))])), elinewidth=0.75, linewidth=0,   marker=".", label = f"σ = $\Gamma ^2 $ x {np.std((diff_phot[t][filtre][r][np.logical_not(np.isnan(diff_phot[t][filtre][r]))]))}")
                            axis[i][j].legend()
                            axis[i][j].set(xlabel='Time JD-2460000', ylabel='Phot. Diff.')
                            axis[i][j].set_title("Filter "+diferent_filters[i] + " comp. "+ diff_phot_names_comp[t]['filter_' + diferent_filters[i]][g][0] + " and ref. star " + diff_phot_names_comp[t]['filter_' + diferent_filters[i]][g][1:] +", $\Gamma ^2 $ using"+ l +", σ_stars_"+ n )
   #plt.show()
    
    #plt.savefig('diff_phot_components+error'+str(t)+'.png')

In [ ]:
# Extract outliers
def exctract_outliers(data, min_quantile, max_quantile):
    
    deletepositions=[]
    
    for k in data:
        if k < np.quantile(data, min_quantile) or k > np.quantile(data, max_quantile):                                    
            deletepositions.append(np.where(data == k)[0][0])
            
    return deletepositions

In [ ]:
def exctract_outliers2(data, time, min_quantile, max_quantile):
    deletepositions=[]
    
    df = pd.DataFrame({'X': time, 'Y': data})
    
    X_train, X_test, Y_train, Y_test = train_test_split(time.reshape(-1, 1), data, test_size=0.2, random_state=42)
    
    # Fit the model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, Y_train)

    # Predict on the training set
    df['Predicted'] = model.predict(time.reshape(-1, 1))
    
    # Calculate residuals
    df['Residuals'] = df['Y'] - df['Predicted']

    # Standardize residuals
    df['StdResiduals'] = (df['Residuals'] - df['Residuals'].mean()) / df['Residuals'].std()

    # Identify outliers using Z-score method (threshold |Z| > 2 for this example)
    outliers = df[np.abs(df['StdResiduals']) > 2]
    
    for k in outliers["Y"]:
        deletepositions.append(np.where(data == k)[0][0])
    # print(len(df['Predicted']), len(data), len(time))
    return deletepositions, df['Predicted']

In [ ]:
def __data_time_binning(data, time, mjd_ini, mjd_end, time_interval_hours, parameter, parameter_error):
    """
    Parameters
    ----------
    data: panda dataframe. Mandatory to have columns [time,parameter,parameter_error] i.e. ['MJD','MAG','eMAG']
    time: column time name 
    mjd_ini/mjd_end: MJD initial/final float
    time_interval_hours: time interval units hours float
    parameter: column parameter name to do stats string
    parameter_error: column parameter name with individual errors string
    
    
    Return
    ----------
    result_df: panda df [time+'_min', time+'_max', time+'_median',time+'std',time+'_n', parameter+'_median', parameter+'_std', parameter+'_n',time+'_central','e'+time]
    """    
    MJD_bins=int((mjd_end-mjd_ini)*24/time_interval_hours)
    
    x_median,y_median,n_x,n_y = __binXY(data[time],data[parameter],statistic='median',xbins=MJD_bins,xrange=None)
    # print(x_median,y_median,n_x,n_y)
    x_std,y_std,n_x,n_y = __binXY(data[time],data[parameter],statistic='std',xbins=MJD_bins,xrange=None)
    x_min,y_min,n_x,n_y = __binXY(data[time],data[parameter],statistic='min',xbins=MJD_bins,xrange=None)
    x_max,y_max,n_x,n_y = __binXY(data[time],data[parameter],statistic='max',xbins=MJD_bins,xrange=None)
    data_bin=pd.DataFrame({time+'_min': x_min, time+'_max': x_max, time+'_median': x_median, time+'_std': x_std, time+'_n': n_x, parameter+'_median': y_median, parameter+'_std': y_std, parameter+'_n': n_y})
    data_bin = data_bin.dropna() # dropping NaN lines
    data_bin['e'+parameter] = data_bin[parameter+'_std'] / np.sqrt(data_bin[parameter+'_n'])
    data_bin[time+'_central']=data_bin[time+'_min']+(data_bin[time+'_max']-data_bin[time+'_min'])/2
    data_bin['e'+time]=(data_bin[time+'_max']-data_bin[time+'_min'])/2
    return(data_bin)

In [ ]:
def __binXY(x,y,statistic='mean',xbins=10,xrange=None):
    """
    Finds statistical value of x and y values in each x bin. 
    Returns the same type of statistic for both x and y.
    See scipy.stats.binned_statistic() for options.
    
    Parameters
    ----------
    x : array
        x values.
    y : array
        y values.
    statistic : string or callable, optional
        See documentation for scipy.stats.binned_statistic(). Default is mean.
    xbins : int or sequence of scalars, optional
        If xbins is an integer, it is the number of equal bins within xrange.
        If xbins is an array, then it is the location of xbin edges, similar
        to definitions used by np.histogram. Default is 10 bins.
        All but the last (righthand-most) bin is half-open. In other words, if 
        bins is [1, 2, 3, 4], then the first bin is [1, 2) (including 1, but 
        excluding 2) and the second [2, 3). The last bin, however, is [3, 4], 
        which includes 4.    
        
    xrange : (float, float) or [(float, float)], optional
        The lower and upper range of the bins. If not provided, range is 
        simply (x.min(), x.max()). Values outside the range are ignored.
    
    Returns
    -------
    x_stat : array
        The x statistic (e.g. mean) in each bin. 
    y_stat : array
        The y statistic (e.g. mean) in each bin.       
    n_x : array of dtype int
        The count of x values in each bin.
    n_y : array of dtype int
        The count of y values in each bin.        
        """
    x_stat, xbin_edges, binnumber = stats.binned_statistic(x, x, 
                                 statistic=statistic, bins=xbins, range=xrange)
    #print(x_stat)
    y_stat, xbin_edges, binnumber = stats.binned_statistic(x, y, 
                                 statistic=statistic, bins=xbins, range=xrange)
    n_x, xbin_edges, binnumber = stats.binned_statistic(x, x, 
                                 statistic='count', bins=xbins, range=xrange)
    n_y, xbin_edges, binnumber = stats.binned_statistic(x, y, 
                                 statistic='count', bins=xbins, range=xrange)
            
    return x_stat, y_stat, n_x, n_y

In [ ]:
# PLOTS COMPONENTS + STARS + SCALE FACTORS, substract outliers and binning (improve it)
import copy
from matplotlib.ticker import MaxNLocator
import statistics as stat
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
a=[250,400,250,100]
for o, t in enumerate(scale_factors_namesA):
    #print(len(scale_factors_namesA), len(scale_factors_namesB))
    
    figure, axis = plt.subplots(len(scale_factors_namesA[t]), int(len(scale_factors_namesA[t]["filter_Lum"])),figsize=(a[o], 60))
    figure.tight_layout(pad=3.5)
    for i, filtre in enumerate(scale_factors_namesA[t]):
        #print(len(scale_factors_namesA[t]), len(scale_factors_namesB[t]))
        for j, l in enumerate(scale_factors_namesA[t][filtre]):
            #print(len(scale_factors_namesA[t][filtre]), len(scale_factors_namesB[t][filtre]))
            for r, n in enumerate(diff_phot_names[t]['filter_' + diferent_filters[i]]):
                #print(len(scale_factors_namesB[t][filtre][j]), len(scale_factors_namesB[t][filtre][j]))
                if n==l[1:]:
                    for g, phot in enumerate(diff_phot_compA[t]['filter_' + diferent_filters[i]]):
                        
                        if l[0]==diff_phot_names_compA[t]['filter_' + diferent_filters[i]][g][0] and  diff_phot_names_compA[t]['filter_' + diferent_filters[i]][g][1:]==n[1:]: 
                            
                            photA = copy.deepcopy(phot)
                            datesA = copy.deepcopy(diff_phot_dates_compA[t]['filter_' + diferent_filters[i]][g])
                            sfA = copy.deepcopy(scale_factorsA[t][filtre][j])
                            
                            deletepositions0 = exctract_outliers(data = photA,  min_quantile = 0.03, max_quantile = 0.97)
                            photA = np.delete(photA, deletepositions0)
                            datesA = np.delete(datesA, deletepositions0)
                            sfA = np.delete(sfA, deletepositions0)
                            
                            #deletepositions, pred1 = exctract_outliers2(data = photA, time = datesA,  min_quantile = 0.03, max_quantile = 0.97)
                            # print(len(deletepositions), len(pred1), len(datesA), len(photA))
                            #photA = np.delete(photA, deletepositions)
                            #datesA = np.delete(datesA, deletepositions)
                            #sfA = np.delete(sfA, deletepositions)
                            #pred1 = np.delete(pred1, deletepositions)
                        
                            
                            photB = copy.deepcopy(diff_phot_compB[t]['filter_' + diferent_filters[i]][g])
                            datesB = copy.deepcopy(diff_phot_dates_compB[t]['filter_' + diferent_filters[i]][g])
                            sfB = copy.deepcopy(scale_factorsB[t][filtre][j])
                              
                            deletepositions02 = exctract_outliers(data = diff_phot_compB[t]['filter_' + diferent_filters[i]][g],min_quantile = 0.03, max_quantile = 0.97)
                            photB = np.delete(photB, deletepositions02)
                            datesB = np.delete(datesB, deletepositions02)     
                            sfB = np.delete(sfB, deletepositions02)
                            
                            #deletepositions2, pred2 = exctract_outliers2(data = photB, time = datesB, min_quantile = 0.03, max_quantile = 0.97)
                            # print(len(pred2), len(datesB), len(photB))                     
                            #photB = np.delete(photB, deletepositions2)
                            #datesB = np.delete(datesB, deletepositions2)     
                            #sfB = np.delete(sfB, deletepositions2)  
                            #pred2 = np.delete(pred2, deletepositions2)
                            # print(len(pred2), len(datesB), len(photB))
                            
                            #deletepositions3=[]
                            #for u, x in enumerate(sfA):
                            #    if x > (stat.mode(sfA)+stat.mode(sfA)/2):# or x < (stat.mode(sfA)-stat.mode(sfA)/2):
                            #        deletepositions3.append(u)
                                                                       
                            #photA = np.delete(photA, deletepositions3)
                            #datesA = np.delete(datesA, deletepositions3)
                            #sfA = np.delete(sfA, deletepositions3)
                            #pred1 = np.delete(pred1, deletepositions3)

                            
                            #deletepositions4=[]
                            #for u, x in enumerate(sfB):
                           #     if x > (stat.mode(sfB)+stat.mode(sfB)/2):# or x < (stat.mode(sfB)-stat.mode(sfB)):
                           #         deletepositions4.append(u)
                            #photB = np.delete(photB, deletepositions4)
                            #datesB = np.delete(datesB, deletepositions4)
                            #sfB = np.delete(sfB, deletepositions4)
                           # pred2 = np.delete(pred2, deletepositions4)

                        
                            err=diff_phot[t][filtre][r][np.logical_not(np.isnan(diff_phot[t][filtre][r]))]
                            #deletepositions5=[]
                            #for k in diff_phot[t][filtre][r][np.logical_not(np.isnan(diff_phot[t][filtre][r]))]:
                                #if k < np.quantile(err, 0.1) or k > np.quantile(err, 0.9):     
                                    #deletepositions5.append(np.where(err == k)[0][0]) 
                            
                            deletepositions5 = exctract_outliers(data = diff_phot[t][filtre][r][np.logical_not(np.isnan(diff_phot[t][filtre][r]))], min_quantile = 0.1, max_quantile = 0.9)
                            
                            err = np.delete(err, deletepositions5)
                            #(datesA, photA, sfA*np.std(err))
                            if len(sfA*np.std(err))>1 and  len(sfB*np.std(err))>1:
                                lista3= zip(datesA, photA, sfA*np.std(err))
                                # print(type(lista3))
                                lista4= zip(datesB, photB, sfB*np.std(err))
                                dfA = pd.DataFrame(lista3, columns=['MJD','MAG','eMAG'])
                                dfB = pd.DataFrame(lista4, columns=['MJD','MAG','eMAG'])
                                    #print(dfA)

                                data_binA = __data_time_binning(data=dfA, time='MJD', mjd_ini=min(datesA), mjd_end=max(datesA), time_interval_hours=24.0, parameter='MAG', parameter_error='eMAG')
                                data_binB = __data_time_binning(data=dfB, time='MJD', mjd_ini=min(datesB), mjd_end=max(datesB), time_interval_hours=24.0, parameter='MAG', parameter_error='eMAG')

                                # datesA2, photA2, ephotA2 = bin_flux_data_julian(photA, datesA, 24)
                                # datesB2, photB2, ephotB2 = bin_flux_data_julian(photB, datesB, 24)
                                # datesA3, sfA2, ee = bin_flux_data_julian(sfA, datesA, 24)
                                # datesB3, sfB2, ee = bin_flux_data_julian(sfB, datesB, 24)

                                if i == 1:
                                    if j==0:
                                        finalA=data_binA["MAG_median"]
                                        finalB=data_binB["MAG_median"]
                                        finaldataA=data_binA["MJD_central"]
                                        finaldataB=data_binB["MJD_central"]

                                # for q,w in enumerate(ephotA2):
                                #     if w==0:
                                #         ephotA2[q]=sfA2[q]*np.std((err))
                                # for q,w in enumerate(ephotB2):
                                #     if w==0:
                                #         ephotB2[q]=sfB2[q]*np.std((err))  pred1
                                # print(photA)
                                axis[i][j].errorbar((datesA)-2460000.0, photA, yerr=sfA*np.std((err)), elinewidth=0.75, linewidth=0,   marker=".", label = f"Component A ($σA _i = \Gamma ^{2}_i $ x {np.std((err))})") # ({np.mean(sfA)})*np.std((diff_phot[t][filtre][r][np.logical_not(np.isnan(diff_phot[t][filtre][r]))]))
                                axis[i][j].errorbar((datesB)-2460000.0, photB, yerr=sfB*np.std((err)), elinewidth=0.75, linewidth=0,   marker=".", label = f"Component B ($σB _i = \Gamma ^{2}_i $ x {np.std((err))})") #({np.mean(sfB)})*np.std((diff_phot[t][filtre][r][np.logical_not(np.isnan(diff_phot[t][filtre][r]))]))

                                # axis[i][j].errorbar((data_binA["MJD_central"])-2460000.0, data_binA["MAG_median"], xerr=data_binA["eMJD"], yerr=data_binA["eMAG"], elinewidth=0.75, linewidth=0, ms=10,  marker=".", mfc="black",mec="black", ecolor="black")#, label = f"$σA _i = \Gamma ^{2}_i $ x {np.std((diff_phot[t][filtre][r][np.logical_not(np.isnan(diff_phot[t][filtre][r]))]))}") # ({np.mean(sfA)})*np.std((diff_phot[t][filtre][r][np.logical_not(np.isnan(diff_phot[t][filtre][r]))]))
                                # axis[i][j].errorbar((data_binB["MJD_central"])-2460000.0, data_binB["MAG_median"], xerr=data_binB["eMJD"], yerr=data_binB["eMAG"], elinewidth=0.75, linewidth=0, ms=10,  marker=".", mfc="black",mec="black", ecolor="black")

                                axis[i][j].locator_params(axis='x', nbins=50)

                                axis[i][j].legend()
                                axis[i][j].set(xlabel='Time JD-2460000', ylabel='Phot. Diff.')
                                axis[i][j].set_title("Filter "+diferent_filters[i] + " comp. "+ diff_phot_names_compA[t]['filter_' + diferent_filters[i]][g][0]+" and " + diff_phot_names_compB[t]['filter_' + diferent_filters[i]][g][0]+ " and ref. star " + diff_phot_names_compA[t]['filter_' + diferent_filters[i]][g][1:] +", $\Gamma ^2 $ using "+ l+ " and "+ scale_factors_namesB[t][filtre][j] +", σ_stars_"+ n ) #" and " + diff_phot_names_compB[t]['filter_' + diferent_filters[i]][g][0]
    
    #plt.savefig('yes'+'.jpeg')
    
    # plt.savefig('diff_phot_components_error_binning'+str(t)+'.jpeg')
    